<a href="https://colab.research.google.com/github/julialromero/regional-representations-graph-model/blob/master/dataset/earth_engine/download_landsat_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Rf4h4kvxU8Pd37zPyTlGCB5gfhARZToiXn8XCwbpAIg&tc=IH5gNsDZPABatBZNCHwhGTTDnozcwr3S35dXjAdJSlY&cc=eb4L4oIdAC3PApxy85rMA9no1RjEh1fzas287F4rmpk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh45l6FIbdHKQ8v0_ZpKUOrbsTuS4WXFjkziqsVQle7VBWIFIa2xMqM

Successfully saved authorization token.


In [3]:
! pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 9.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [4]:
! pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 53.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [5]:
import os
import ee
import geemap
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')
output_folder = r'/content/drive/MyDrive/landsat/download_landsat_images/test1'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import geopandas as gpd
import pandas as pd
tract_data_dir = r'/content/drive/MyDrive/landsat/nyc_metro_boundaries/nyc_metro_boundaries.shp'

df = gpd.read_file(tract_data_dir)
df.rename(columns={'GEOID_TRAC': 'GEOID'}, inplace=True)  
# print(df.head())
shapefile = df[['GEOID', 'geometry']] 


shapefile.iloc[0:100].to_file("/content/drive/MyDrive/landsat/nyc_metro_boundaries/test.shp")

In [7]:
Map = geemap.Map()
tracts = geemap.shp_to_ee('/content/drive/MyDrive/landsat/nyc_metro_boundaries/test.shp')

In [11]:
import os
def createDir(dirPath):
  try:
    os.makedirs(dirPath)
  except:
    pass

In [12]:
save_dir = 'drive/MyDrive/landsat/download_landsat_images/'
tlist = tracts.toList(100)

for i in range(0, tlist.length().getInfo()):
  subdir = f"{save_dir}{tlist.get(i).getInfo()['properties']['GEOID']}"
  if not os.path.exists(subdir):
    createDir(subdir)
  feat = ee.Feature(tlist.get(i))
  roi = feat.geometry()

  collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
      .filter(ee.Filter.contains('.geo', roi))
      .filterDate('2020-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
  )
  top = collection.limit(10)

  for img_idx in top.aggregate_array('system:index').getInfo():
    img = ee.Image('LANDSAT/LC08/C02/T1_TOA/' + img_idx)
    filename = f"{subdir}/{img_idx}.tif"

    new_img = img.clip(roi)

    geemap.ee_export_image(
        new_img, filename=filename, scale=90, region=roi, file_per_band=True
    )

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat/download_landsat_images/09003514500
Generating URL ...
Please wait .

In [ ]:
# plot all
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(new_img, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

## Part 2: Define a dataset object

In [ ]:
 ! pip install torchgeo

In [113]:
import torch

In [115]:
import torchgeo
from torch.utils.data import DataLoader
from torchgeo.datasets import RasterDataset, stack_samples, unbind_samples
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

In [116]:
dir_path = 'content/drive/MyDrive/landsat/download_landsat_images/test1/09003514500_0.tif'
class Landsat8(RasterDataset):
    filename_glob = "T*_B02_10m.tif"
    filename_regex = r"^.{6}_(?P<date>\d{8}T\d{6})_(?P<band>B0[\d])"
    date_format = "%Y%m%dT%H%M%S"
    is_image = True
    separate_files = True
    all_bands = ["B02", "B03", "B04", "B08"]
    rgb_bands = ["B04", "B03", "B02"]

    def plot(self, sample):
        # Find the correct band index order
        rgb_indices = []
        for band in self.rgb_bands:
            rgb_indices.append(self.all_bands.index(band))

        # Reorder and rescale the image
        image = sample["image"][rgb_indices].permute(1, 2, 0)
        image = torch.clamp(image / 10000, min=0, max=1).numpy()

        # Plot the image
        fig, ax = plt.subplots()
        ax.imshow(image)

        return fig

SyntaxError: ignored